# Sentiment Analysis on Product Reviews using Multinomial Naive Bayes Algorithm 

This is the step by step process from Data Preprocessing to Data Visualization. Note that the reviews were scraped from Amazon. 

### Importing necessary libraries

In [66]:
import pandas as pd
import numpy as np
import re
from nltk.sentiment.vader import SentimentIntensityAnalyzer
import nltk
from nltk.corpus import stopwords   
nltk.download('stopwords')
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords
stop = stopwords.words('english')
stemmer = SnowballStemmer("english")
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from tabulate import tabulate
import plotly.express as px

vectorizer = CountVectorizer()

[nltk_data] Downloading package stopwords to C:\Users\Karl Jade C.
[nltk_data]     Silva\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### Importing the dataset 

In [67]:
df = pd.read_csv("D:/desktop/Daily Routine/Portfolio's Content/3. Sentiment Analysis on Product Reviews using Multinomial Naive Bayes Algorithm/Dataset/1.Scraped Reviews.csv")
df.head(1)

,Unnamed: 0,username,verification,reviewpage,name,categoryRank,category,cost,reviewbody,reviewstar,product_url,product_name,review_page_url,no_of_reviews
0,0,"[{'review_username': 'Craig'}, {'review_userna...","[{'review_verification': 'Verified Purchase'},...",https://www.amazon.com/product-reviews/B08BJDF...,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,Desktops,$266.99,"[{'review_body': ""So far I have only used this...",[{'review_star': 'a-icon a-icon-star a-star-5 ...,https://www.amazon.com/Dell-Optiplex-9020-Disp...,Dell Optiplex 9020 Small Form Factor Desktop w...,https://www.amazon.com/product-reviews/B08BJDF...,NaN


## 1. Splitting Process

Orginally, the scraped product reviews are merged into one cell. Hence, the reviews undergone splitting process.

In [68]:
# STORE DUPLICATES IN A VARIABLE
duplicateRows = df[df.duplicated(['reviewbody'])]

# Dropping the duplicated reviews and the keep the last one

df.drop_duplicates(subset="reviewbody",keep='last', inplace=True)
df = df.reset_index(drop=True)
df.head(1)

,Unnamed: 0,username,verification,reviewpage,name,categoryRank,category,cost,reviewbody,reviewstar,product_url,product_name,review_page_url,no_of_reviews
0,0,"[{'review_username': 'Craig'}, {'review_userna...","[{'review_verification': 'Verified Purchase'},...",https://www.amazon.com/product-reviews/B08BJDF...,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,Desktops,$266.99,"[{'review_body': ""So far I have only used this...",[{'review_star': 'a-icon a-icon-star a-star-5 ...,https://www.amazon.com/Dell-Optiplex-9020-Disp...,Dell Optiplex 9020 Small Form Factor Desktop w...,https://www.amazon.com/product-reviews/B08BJDF...,NaN


In [69]:
# Filtering and Selecting the needed columns 
dfdata = df.filter(['username', 'verification', 'product_name', 'reviewbody', 'reviewstar', 'categoryRank', 'category'])
dfdata.head(1)

,username,verification,product_name,reviewbody,reviewstar,categoryRank,category
0,"[{'review_username': 'Craig'}, {'review_userna...","[{'review_verification': 'Verified Purchase'},...",Dell Optiplex 9020 Small Form Factor Desktop w...,"[{'review_body': ""So far I have only used this...",[{'review_star': 'a-icon a-icon-star a-star-5 ...,#1,Desktops


In [70]:
# Removing the first and last characters of a string on a single cell and adding comma to ease the replace code later 

dfdata['username'] = dfdata['username'].str[1:-1] 
dfdata['username'] = dfdata['username']+','

dfdata['username'] = dfdata['username'].str.replace(r"[\"]", r"'")
dfdata['username'] = dfdata['username'].str.replace("'},", '')

dfdata['verification'] = dfdata['verification'].str[1:-1]
dfdata['verification'] = dfdata['verification']+','


dfdata['verification'] = dfdata['verification'].str.replace(r"[\"]", r"'")
dfdata['verification'] = dfdata['verification'].str.replace("'},", '')

dfdata['reviewbody'] = dfdata['reviewbody'].str[1:-1]
dfdata['reviewbody'] = dfdata['reviewbody']+','

dfdata['reviewbody'] = dfdata['reviewbody'].str.replace(r"[\"]", r"'")

dfdata['reviewbody'] = dfdata['reviewbody'].str.replace("'},", '')

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\4025647722.py:6: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\4025647722.py:7: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\4025647722.py:13: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\4025647722.py:14: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\4025647722.py:19: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\4025647722.py:21: FutureWarning:

The default value of regex

In [71]:
# 1. Create an empty dataframes to be used for saving the splitted reviews

main_main_df = pd.DataFrame(columns=['category','product_name','username','verification','reviews','review_stars','categoryRank	'])
main_cat_df = pd.DataFrame(columns=['category'])
main_prod_df = pd.DataFrame(columns=['product_name'])
main_review_df = pd.DataFrame(columns=['reviews'])
main_star_df = pd.DataFrame(columns=['review_stars'])
main_verification_df = pd.DataFrame(columns=['verification'])
main_user_df = pd.DataFrame(columns=['username'])
main_rank_df = pd.DataFrame(columns=['rank'])

# 2.Creating a for loop to split the reviews per column 

x = 0

for i in dfdata.index:
    if x <= len(dfdata):
        
        # 1.RANK

        rank1 = [0,0,0,0,0,0,0,0,0,0]
        rank1 = pd.DataFrame(rank1, columns=['rank'])
        rank1['rank'] = dfdata.iat[x,5]
        
        main_rank_df = pd.concat([main_rank_df, rank1])
        main_rank_df = main_rank_df.reset_index(drop=True)
        

        # 2.CATEGORY

        cat1 = [0,0,0,0,0,0,0,0,0,0]
        cat1 = pd.DataFrame(cat1, columns=['category'])
        cat1['category'] = dfdata.iat[x,6]
        
        main_cat_df = pd.concat([main_cat_df, cat1])
        main_cat_df = main_cat_df.reset_index(drop=True)
        
        
        # 3. PRODUCT NAME

        prod1 = [0,0,0,0,0,0,0,0,0,0]
        prod1 = pd.DataFrame(prod1, columns=['product_name'])
        prod1['product_name'] = dfdata.iat[x,2]

        main_prod_df = pd.concat([main_prod_df, prod1])
        main_prod_df = main_prod_df.reset_index(drop=True)

        # 4.USERNAME

        user1 = dfdata.iat[x,0]
        user2 = user1.split("{'review_username': '")
        user3 = pd.DataFrame(user2)
        user4 = user3.drop(0)
        user4.rename(columns={0:'username'}, inplace = True)
        user4 = user4.reset_index(drop=True)

        main_user_df = pd.concat([main_user_df, user4])
        main_user_df = main_user_df.reset_index(drop=True)


        # 5.VERIFICATION

        verification1 = dfdata.iat[x,1]
        verification2 = verification1.split("{'review_verification': '")
        verification3 = pd.DataFrame(verification2)
        verification4 = verification3.drop(0)
        verification4.rename(columns={0:'verification'}, inplace = True)
        verification4 = verification4.reset_index(drop=True)

        
        main_verification_df = pd.concat([main_verification_df, verification4])
        main_verification_df = main_verification_df.reset_index(drop=True)

        # 6. REVIEWS

        words1 = dfdata.iat[x,3]
        words2 = words1.split("{'review_body': '")
        words3 = pd.DataFrame(words2)
        words4 = words3.drop(0)
        words4.rename(columns={0:'reviews'}, inplace = True)
        words4 = words4.reset_index(drop=True)

        
        main_review_df = pd.concat([main_review_df, words4])
        main_review_df = main_review_df.reset_index(drop=True)

        # 7. STARS

        star = dfdata.iat[x,4]
        star1 = re.sub('[^A-Za-z0-9_]+', ' ', star)
        star1 = star1.replace("a icon a icon star a star 5 review rating ","1.0")
        star1 = star1.replace("a icon a icon star a star 4 review rating ","0.5")
        star1 = star1.replace("a icon a icon star a star 3 review rating ","0")
        star1 = star1.replace("a icon a icon star a star 2 review rating ","-0.5")
        star1 = star1.replace("a icon a icon star a star 1 review rating ","-1.0")
        star2 = star1.split('review_star')
        star3 = pd.DataFrame(star2)
        star4 = star3.drop(0)
        star4.rename(columns={0:'review_stars'}, inplace = True)
        star4 = star4.reset_index(drop=True)
        
        x += 1
        main_star_df = pd.concat([main_star_df, star4])
        main_star_df = main_star_df.reset_index(drop=True)


main_main_df = pd.concat([main_user_df,main_verification_df,main_cat_df,main_prod_df,main_rank_df,main_review_df,main_star_df], axis = 1)
main_main_df = main_main_df.reset_index(drop=True)
main_main_df['reviews'].replace('', np.nan, inplace=True)
main_main_df['review_stars'].replace('', np.nan, inplace=True)
main_main_df['username'].replace('', np.nan, inplace=True)
main_main_df['verification'].replace(' ', np.nan, inplace=True)

In [72]:
# Checking the output

final_data = main_main_df
final_data.head()
len(final_data)


31060

After the splitting process, a total of 31,060 reviews were obtained. However, these reviews had not undergone duplication removal at that point. Hence, a count was performed, and the duplicated reviews in the current dataset were removed. The next step followed thereafter.

## 2. Removing the Duplicated Reviews


In [73]:
# Saving the duplicated reviews in duplicateRows variable 

duplicateRows = final_data[final_data.duplicated(['reviews'])]
len(duplicateRows)

927

In [74]:
final_data.drop_duplicates(subset="reviews",keep='last', inplace=True)
len(final_data)

30133

The analysis revealed that there were 927 duplicated reviews in the dataset. Consequently, these duplicates were promptly removed, resulting in a final count of 30,133 reviews that were deemed ready for the preliminary text cleaning process.

## 3. Preliminary Text-preprocessing (Removal of URLs, Newlines, Special Characters)

In [75]:
#Removal of URLs
final_data['reviews'] = final_data['reviews'].str.replace('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', '')

#Removal of Newlines
final_data['reviews'].replace(to_replace=[r"\\t|\\n|\\r", "\t|\n|\r"], value=["",""], regex=True, inplace=True)

#Removal of Special Characters
final_data['reviews'] = final_data['reviews'].str.replace("&", '').str.replace("%", '').str.replace("$", '').str.replace("#", '').str.replace("@", '')

final_data['reviews'].head(1)

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\3005413119.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\3005413119.py:8: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



0    So far I have only used this product for a ver...
Name: reviews, dtype: object

## 4. Computing Sentiment Score using Vader and Getting Difference between Product Rating and Sentiment Score

In [76]:
path = pd.read_csv("D:/desktop/Daily Routine/Portfolio's Content/3. Sentiment Analysis on Product Reviews using Multinomial Naive Bayes Algorithm/Dataset/2.Classified Reviews.csv")
main_main_df = path

In [77]:
#Creating a new variable 
main_main_df4 = main_main_df
main_main_df4

#Creating the instance of Sentiment Intensity Analyzer

sent = SentimentIntensityAnalyzer()

polarity = [round(sent.polarity_scores(i)['compound'], 2) for i in main_main_df4['reviews']]
main_main_df4['sentiment_score'] = polarity
main_main_df4['review_stars'] = pd.to_numeric(main_main_df4['review_stars'])
main_main_df4['difference'] = main_main_df4['review_stars'] - main_main_df4['sentiment_score']

main_main_df4.head()

,Unnamed: 0,username,verification,category,product_name,rank,reviews,review_stars,sentiment_score,difference
0,0,Craig,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,So far I have only used this product for a ver...,1.0,0.97,0.03
1,1,J. Rodriguez,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,"I have got to say, I could not be more pleased...",1.0,0.84,0.16
2,2,Mark,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,So this is the first time I bought a ' refurbi...,0.5,0.82,-0.32
3,3,jordan kelley,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,Great product if all you are needing to do is ...,0.5,0.37,0.13
4,4,,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,"I bought it for day trading, couldn't be happi...",1.0,0.96,0.04


## 5. Classifying Spam and Ham Reviews and Labeling of Reviews 

In [78]:
condition = [
    (main_main_df4['difference'] >= 0.5),
    (main_main_df4['difference'] < 0.5)
]
values = ['spam','ham']
main_main_df4['classification'] = np.select(condition, values)

condition = [
    (main_main_df4['sentiment_score'] > 0),
    (main_main_df4['sentiment_score'] == 0),
    (main_main_df4['sentiment_score'] < 0)
]
values = ['positive','neutral','negative']
main_main_df4['product_sentiment'] = np.select(condition, values)
main_main_df4.head()

,Unnamed: 0,username,verification,category,product_name,rank,reviews,review_stars,sentiment_score,difference,classification,product_sentiment
0,0,Craig,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,So far I have only used this product for a ver...,1.0,0.97,0.03,ham,positive
1,1,J. Rodriguez,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,"I have got to say, I could not be more pleased...",1.0,0.84,0.16,ham,positive
2,2,Mark,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,So this is the first time I bought a ' refurbi...,0.5,0.82,-0.32,ham,positive
3,3,jordan kelley,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,Great product if all you are needing to do is ...,0.5,0.37,0.13,ham,positive
4,4,,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,"I bought it for day trading, couldn't be happi...",1.0,0.96,0.04,ham,positive


## 6. Final Text Cleaning (Removal of Special Characters, numbers, emojies, stopwords, whitespaces, text normalization, word stemming)

In [79]:
#Get only the ham reviews for training the model

hamdf = main_main_df4[main_main_df4['classification'] == 'ham']
data = hamdf

In [80]:
# Removal of Special Characters 
data.reviews = data['reviews'].str.replace('[^a-zA-Z]', ' ') 

# Removal of Stopwords
data['reviews'] = data['reviews'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
data["reviews"]= data["reviews"].astype(str).str.split(" ", expand = False)

# Removal of Whitespaces 
data["reviews"]= data["reviews"].str.join(" ")

# Text Normalization 
data["reviews"] = data["reviews"].str.lower()

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\712204869.py:2: FutureWarning:

The default value of regex will change from True to False in a future version.

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\712204869.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\712204869.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\712204869.py:6: SettingWithC

In [81]:
# Peforming Word Stemming 

data["reviews"]= data["reviews"].astype(str).str.split(" ", expand = False)
data["reviews"] = data["reviews"].apply(lambda x: [stemmer.stem(y) for y in x])
data["reviews"]= data["reviews"].str.join(" ")
data.head()

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\667865257.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\667865257.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\Karl Jade C. Silva\AppData\Local\Temp\ipykernel_8528\667865257.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation

,Unnamed: 0,username,verification,category,product_name,rank,reviews,review_stars,sentiment_score,difference,classification,product_sentiment
0,0,Craig,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,so far i use product short period time i fair ...,1.0,0.97,0.03,ham,positive
1,1,J. Rodriguez,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,i got say i could pleas comput i game main use...,1.0,0.84,0.16,ham,positive
2,2,Mark,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,so first time i bought refurbish comput i some...,0.5,0.82,-0.32,ham,positive
3,3,jordan kelley,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,great product need basic comput mayb game i ne...,0.5,0.37,0.13,ham,positive
4,4,,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,i bought day trade happier i big fan refurbish...,1.0,0.96,0.04,ham,positive


In [82]:
# Creating new variable where neutral is dropped. 
newdata = data[data.product_sentiment != 'neutral']
newdata.product_sentiment.value_counts()
data = newdata
data.product_sentiment.value_counts()

positive    18297
negative     2873
Name: product_sentiment, dtype: int64

After running the prelimary to final text cleaning, we obtained the following: 

Positive - 18297 Negative - 2873 Neutral - 734

We drop the neutral reviews because only the positive and negative reviews only is needed for the training. We can now start model building and testing.

## 7. Model Building

Importing the dataset 

In [83]:
path = pd.read_csv("D:/desktop/Daily Routine/Portfolio's Content/3. Sentiment Analysis on Product Reviews using Multinomial Naive Bayes Algorithm/Dataset/3.Labeled Reviews.csv")
model_data = (path)

In [84]:
model_data.head(1)

,Unnamed: 0,username,verification,category,product_name,rank,reviews,review_stars,sentiment_score,difference,classification,product_sentiment
0,0,Craig,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,so far i use product short period time i fair ...,1.0,0.97,0.03,ham,positive


1. Converting the sentiments into binary, 0 - Positive and 1 = Negative. This will be used for training

In [85]:
condition = [
    (model_data['product_sentiment'] == 'positive'),
    (model_data['product_sentiment'] == 'negative')
]
values = [0,1]
model_data['biproduct_sentiment'] = np.select(condition, values)
model_data.head(1)

,Unnamed: 0,username,verification,category,product_name,rank,reviews,review_stars,sentiment_score,difference,classification,product_sentiment,biproduct_sentiment
0,0,Craig,Verified Purchase,Desktops,Dell Optiplex 9020 Small Form Factor Desktop w...,#1,so far i use product short period time i fair ...,1.0,0.97,0.03,ham,positive,0


2. Converting the binary product sentiments to numeric datatype

In [86]:
model_data["biproduct_sentiment"] = model_data["biproduct_sentiment"].apply(pd.to_numeric)
model_data.dtypes

Unnamed: 0               int64
username                object
verification            object
category                object
product_name            object
rank                    object
reviews                 object
review_stars           float64
sentiment_score        float64
difference             float64
classification          object
product_sentiment       object
biproduct_sentiment      int64
dtype: object

3. Checking the number of positive and negative reviews 

In [87]:
model_data.biproduct_sentiment.value_counts()

0    18297
1     2873
Name: biproduct_sentiment, dtype: int64

4. Training the model

In [88]:
# Create a vectorizer

vectorizer = CountVectorizer()
matrix = vectorizer.fit_transform(model_data.reviews)

X = matrix
y = model_data.biproduct_sentiment

# Splitting the dataset into training and testing set

X_train, X_test, y_train, y_test = train_test_split(X, y)

# Train the classifier using the training dataset

classifier = MultinomialNB()
classifier.fit(X_train, y_train)

MultinomialNB()

5. Testing the model

In [89]:
y_true = y_test
y_pred = classifier.predict(X_test)
matrix = confusion_matrix(y_true, y_pred)

label_names = pd.Series(['Positive', 'Negative'])
pd.DataFrame(matrix,
     columns='Predicted ' + label_names,
     index='Is ' + label_names).div(matrix.sum(axis=1), axis=0)

,Predicted Positive,Predicted Negative
Is Positive,0.940760,0.059240
Is Negative,0.362809,0.637191


6. Running confusion matrix 

In [90]:
pred_nb = classifier.predict(X_test)
confusion = confusion_matrix(y_test, pred_nb)
print("Confusion matrix:\n{}".format(confusion))

Confusion matrix:
[[4256  268]
 [ 279  490]]


7. Running a classification report

In [91]:
matrix = classification_report(y_test, pred_nb)
print('classification_report : \n', matrix)

classification_report : 
               precision    recall  f1-score   support

           0       0.94      0.94      0.94      4524
           1       0.65      0.64      0.64       769

    accuracy                           0.90      5293
   macro avg       0.79      0.79      0.79      5293
weighted avg       0.90      0.90      0.90      5293



## 8. Data Visualization 

### Importing dataset

In [92]:
visualize_data = pd.read_csv("D:/desktop/Daily Routine/Portfolio's Content/3. Sentiment Analysis on Product Reviews using Multinomial Naive Bayes Algorithm/Dataset/4.Data Visualization.csv")
visualize_data.head(1)

,reviews,review_stars,category,sentiment_score,difference,classification,product_sentiment,nb_prediction,bi_nb_prediction
0,far use product short period time fair comput ...,1.0,Desktops,0.97,0.03,ham,positive,1,positive


### Split Dataframes for each category 

In [93]:
# 1.Desktop

desktopdf = visualize_data.loc[(visualize_data['category'] == 'Desktops')]
desktopdf.head()

,reviews,review_stars,category,sentiment_score,difference,classification,product_sentiment,nb_prediction,bi_nb_prediction
0,far use product short period time fair comput ...,1.0,Desktops,0.97,0.03,ham,positive,1,positive
1,got say could plea comput game main use brow k...,1.0,Desktops,0.30,0.70,spam,positive,1,positive
2,first time bought refurbish comput somewhat su...,0.5,Desktops,0.46,0.04,ham,positive,0,negative
3,great product need basic comput mayb game need...,0.5,Desktops,0.62,-0.12,ham,positive,1,positive
4,bought day trade happier big fan refurbish ele...,1.0,Desktops,0.95,0.05,ham,positive,1,positive


In [94]:
# 2.Laptop

laptopdf = visualize_data.loc[(visualize_data['category'] == 'Laptops')]
laptopdf = laptopdf.reset_index(drop=True)
laptopdf.head()

,reviews,review_stars,category,sentiment_score,difference,classification,product_sentiment,nb_prediction,bi_nb_prediction
0,like size comput lightweight touch screen purc...,0.5,Laptops,0.93,-0.43,ham,positive,1,positive
1,product descript th gen processor option cpu t...,0.5,Laptops,0.27,0.23,ham,positive,1,positive
2,bought son colleg love,0.5,Laptops,0.64,-0.14,ham,positive,1,positive
3,laptop coupl day realli like replac acer const...,1.0,Laptops,0.97,0.03,ham,positive,1,positive
4,love came nice,1.0,Laptops,0.79,0.21,ham,positive,1,positive


In [95]:
# 2.Tablet

tabletdf = visualize_data.loc[(visualize_data['category'] == 'Tablets')]
tabletdf = tabletdf.reset_index(drop=True)
tabletdf.head()

,reviews,review_stars,category,sentiment_score,difference,classification,product_sentiment,nb_prediction,bi_nb_prediction
0,year updat need biggest ipad one need great pu...,1.0,Tablets,0.62,0.38,ham,positive,1,positive
1,setup devic straight forward transfer use appl...,0.5,Tablets,-0.20,0.70,spam,negative,1,positive
2,purcha birthday gift husband love,1.0,Tablets,0.80,0.20,ham,positive,1,positive
3,need ipad final decid buy one veri afford ligh...,1.0,Tablets,0.90,0.10,ham,positive,1,positive
4,realli skeptic purcha ipad amazon howev fianc ...,1.0,Tablets,0.90,0.10,ham,positive,1,positive


### Plotting Pie Chart for all categories (Spam and Ham Reviews)

In [96]:
# 1. Spam and Ham Reviews 

series_spam_ham = visualize_data['classification'].value_counts()
spam_ham = pd.DataFrame(series_spam_ham)
spam_ham = spam_ham.reset_index()
spam_ham.columns = ['classification', 'count']
display(spam_ham)

fig = px.pie(spam_ham,
             values='count',
             names='classification')
display(fig)

,classification,count
0,ham,19018
1,spam,2152


### Plotting Bar Chart for all categories (Sentiment Score)

In [97]:
senti_series = visualize_data['sentiment_score'].value_counts()
senti_score = pd.DataFrame(senti_series)
senti_score = senti_score.reset_index()
senti_score.columns = ['sentiment_scores', 'count']
display(senti_score)

fig = px.bar(senti_score, 
             x='sentiment_scores', y='count',
             color = 'sentiment_scores',
             color_continuous_scale=["red", "grey", "blue"],
             range_color = [-1,1],
             opacity= 0.6,
             )
fig.update_traces(marker_line_color='black',
                  marker_line_width=3, opacity=1)
fig.show()

,sentiment_scores,count
0,0.62,1129
1,0.64,1063
2,0.00,874
3,0.85,520
4,0.92,488
...,...,...
193,-0.29,2
194,-0.01,2
195,-0.12,2
196,0.06,1


### Plotting Bar Chart for all categories (Product Sentiments)

In [98]:
series_prod_senti = visualize_data['bi_nb_prediction'].value_counts()
prod_senti = pd.DataFrame(series_prod_senti)
prod_senti = prod_senti.reset_index()
prod_senti.columns = ['product_sentiments', 'count']
display(prod_senti)

fig = px.bar(prod_senti, 
             x='product_sentiments', y='count', 
             color = 'product_sentiments',
             color_discrete_sequence=["blue", "red"]
            )
fig.show()

,product_sentiments,count
0,positive,16634
1,negative,4536


### Potting Bar Graph for Product Sentiments 

In [99]:
# 1. Desktops

series1 = desktopdf['bi_nb_prediction'].value_counts()
df_result1 = pd.DataFrame(series1)
df_result1 = df_result1.reset_index()  
df_result1.columns = ['product_sentiment', 'count']
display(df_result1)

fig = px.bar(df_result1, 
             x='product_sentiment', y='count',
             color = 'product_sentiment',
             color_discrete_sequence=["blue", "red", "grey"])
fig.show()

,product_sentiment,count
0,positive,3267
1,negative,1385


In [100]:
# 2. Laptop

series2 = laptopdf['bi_nb_prediction'].value_counts()
df_result2 = pd.DataFrame(series2)
df_result2 = df_result2.reset_index()  
df_result2.columns = ['product_sentiment', 'count']
display(df_result2)

fig = px.bar(df_result2, 
             x='product_sentiment', y='count',
             color = 'product_sentiment',
             color_discrete_sequence=["blue", "red", "grey"])
fig.show()

,product_sentiment,count
0,positive,5821
1,negative,1843


In [101]:
# 3. Tablet

series3 = tabletdf['bi_nb_prediction'].value_counts()
df_result3 = pd.DataFrame(series3)
df_result3 = df_result3.reset_index()  
df_result3.columns = ['product_sentiment', 'count']
display(df_result3)

fig = px.bar(df_result3, 
             x='product_sentiment', y='count', 
             color = 'product_sentiment',
             color_discrete_sequence=["blue", "red", "grey"],
             )
fig.show()

,product_sentiment,count
0,positive,7546
1,negative,1308


### Potting Bar Graph for Sentiment Scores

In [102]:
# 1.Desktop

series4 = desktopdf['sentiment_score'].value_counts()
df_result4 = pd.DataFrame(series4)
df_result4 = df_result4.reset_index()  
df_result4.columns = ['sentiment_score', 'count']
display(df_result4)

fig = px.bar(df_result4, 
             x='sentiment_score', y='count', 
             color = 'sentiment_score',
             color_continuous_scale=["red", "grey", "blue"],
             range_color = [-1,1],
             opacity= 0.6,
            )
fig.update_traces(marker_line_color='black',
                  marker_line_width=3, opacity=1)
fig.show()

,sentiment_score,count
0,0.62,286
1,0.00,247
2,0.44,110
3,0.64,102
4,0.90,101
...,...,...
176,-0.19,1
177,-0.35,1
178,-0.45,1
179,-0.07,1


In [103]:
# 2.Laptop

series5 = laptopdf['sentiment_score'].value_counts()
df_result5 = pd.DataFrame(series5)
df_result5 = df_result5.reset_index()  
df_result5.columns = ['sentiment_score', 'count']
display(df_result5)

fig = px.bar(df_result5, 
             x='sentiment_score', y='count',
             color = 'sentiment_score',
             color_continuous_scale=["red", "grey", "blue"],
             range_color = [-1,1],
             opacity= 0.6,
             )
fig.update_traces(marker_line_color='black',
                  marker_line_width=3, opacity=1)
fig.show()

,sentiment_score,count
0,0.62,428
1,0.64,361
2,0.00,315
3,0.85,185
4,0.95,175
...,...,...
186,-0.21,1
187,-0.31,1
188,0.29,1
189,0.39,1


In [104]:
# 3.Tablet 

series6 = tabletdf['sentiment_score'].value_counts()
df_result6 = pd.DataFrame(series6)
df_result6 = df_result6.reset_index()  
df_result6.columns = ['sentiment_score', 'count']
display(df_result6)

fig = px.bar(df_result6, 
             x='sentiment_score', y='count',
             color = 'sentiment_score',
             color_continuous_scale=["red", "grey", "blue"],
             range_color = [-1,1],
             opacity= 0.6,
             )
fig.update_traces(marker_line_color='black',
                  marker_line_width=3, opacity=1)
fig.show()

,sentiment_score,count
0,0.64,600
1,0.62,415
2,0.00,312
3,0.77,242
4,0.85,237
...,...,...
183,0.11,1
184,-0.94,1
185,-0.91,1
186,-0.07,1


### Potting Pie Chart for Spam and Ham Reviews 

In [105]:
# 1. Desktop 

desktopclassificationdf = visualize_data.loc[(visualize_data['category'] == 'Desktops')]
desktopclassificationdf = desktopclassificationdf.reset_index(drop=True)

series_desktop = desktopclassificationdf['classification'].value_counts()
df_result_desktop = pd.DataFrame(series_desktop)
df_result_desktop = df_result_desktop.reset_index()
df_result_desktop.columns = ['classification', 'count']
display(df_result_desktop)

fig = px.pie(df_result_desktop,
             values='count',
             names='classification')
display(fig)

,classification,count
0,ham,4153
1,spam,499


In [106]:
# 2. Laptops

laptopclassificationdf = visualize_data.loc[(visualize_data['category'] == 'Laptops')]
laptopclassificationdf = laptopclassificationdf.reset_index(drop=True)

series_laptop = laptopclassificationdf['classification'].value_counts()
df_result_laptop = pd.DataFrame(series_laptop)
df_result_laptop  = df_result_laptop .reset_index()
df_result_laptop.columns = ['classification', 'count']
display(df_result_laptop)

fig = px.pie(df_result_laptop,
             values='count',
             names='classification')
display(fig)

,classification,count
0,ham,6967
1,spam,697


In [107]:
# 3. Tablets

tabletclassificationdf = visualize_data.loc[(visualize_data['category'] == 'Tablets')]
tabletclassificationdf = tabletclassificationdf.reset_index(drop=True)


series_tablet = tabletclassificationdf['classification'].value_counts()
df_result_tablet = pd.DataFrame(series_tablet)
df_result_tablet= df_result_tablet.reset_index()
df_result_tablet.columns = ['classification', 'count']
display(df_result_tablet)

fig = px.pie(df_result_tablet,
             values='count',
             names='classification')
display(fig)

,classification,count
0,ham,7898
1,spam,956
